## IPYNB to add a column coordinates with (long, lat) to the data from the website events.ch

In [1]:
import glob
import numpy as np
import pandas as pd
import LocationUtils as lu
dictionaryFile = "ClubsGeo"

Import all csv from the website events.ch

In [2]:
files = []
files = glob.glob("EventsChData/*.csv")
print(len(files))
ClubDataFrame = pd.read_csv(files[0])

for file in files[1:]:
    df= pd.read_csv(file)
    ClubDataFrame = ClubDataFrame.append(df,ignore_index = True)
    
print(ClubDataFrame.shape)
ClubDataFrame.head()


1456
(36340, 6)


,Unnamed: 0,location,event,date,artists,genre
0,0,Zürich,Andy-Gruber-Band,2016-04-22,['Andy Gruber Band '],traditional
1,1,Basel,Lilian's-White-Party,2016-04-22,['DJ Raykard '],Pop
2,2,Basel,Orgelspiel-zum-Feierabend,2016-04-22,['Martin Sander '],Classic
3,3,Dietikon,Feierabendmusik,2016-04-22,['Bernhard Hörler '],Classic
4,4,Zürich,Oomph,2016-04-22,['Oomph '],Electronic


We have this annoying column because the csv was written without "index=False" (my bad).

In [3]:
ClubDataFrame = ClubDataFrame.drop('Unnamed: 0', 1)

Check if DF is correct now

In [4]:
ClubDataFrame[13:16]

,location,event,date,artists,genre
13,Bern,Schuberts-Tragische,2016-04-22,[' Veronika Eberle '],Classic
14,Basel,Jazzfestival-Basel,2016-04-22,"[' Kenny Garrett ', ' Vernell Brown ', ' Corco...",NaN
15,null,Konzert,2016-04-22,[' Tine Thing Helseth '],Classic


run once cells below to populate the dictionary = create entry from "1020 Renens" and "1020;Renens" to "Renens" to avoid some searches.

In [5]:
dic = lu.loadDictionary("newDic")

In [6]:
print(len(dic))

650


In [7]:
ClubDataFrame = ClubDataFrame[ClubDataFrame.location != "null"]

Check if invalid location are gone

In [8]:
ClubDataFrame[13:16]

,location,event,date,artists,genre
13,Bern,Schuberts-Tragische,2016-04-22,[' Veronika Eberle '],Classic
14,Basel,Jazzfestival-Basel,2016-04-22,"[' Kenny Garrett ', ' Vernell Brown ', ' Corco...",NaN
16,Bern,Oli-Kehrli,2016-04-22,['Oli Kehrli '],Pop


In [9]:
def addCoordinatesColumn(df, dictionary=None, location_column_name="location", coordinates_column_name = "coordinates",replace=False, inplace = False):
    tmp = df.copy()
    if(coordinates_column_name in df.columns):
        if(replace):
            if(inplace):
                df.drop(coordinates_column_name,1, inplace=True)
            else:
                tmp.drop(coordinates_column_name,1, inplace=True)
        else:
            print("column ",coordinates_column_name," already in dataframe.")
            return 
    serie = pd.Series(data = np.nan, index = df.index, dtype = np.dtype)
    for (i,r) in df.iterrows():
        serie[i]= lu.getLocation(df.loc[i,location_column_name], dictionary)[0]
        if(i%128 == 0): 
            print(i)
    if(inplace):
        df.insert(len(df.columns), coordinates_column_name,serie)
    else: 
        tmp.insert(len(tmp.columns), coordinates_column_name,serie)
        return tmp

Add the coordinate column to he dataframe

In [10]:
addCoordinatesColumn(ClubDataFrame, dic, inplace=True)

0
out of bounds:  Baden  :  48.56992809999999 , 8.530397199999998
out of bounds:  Kuppel  :  50.1810945 , 12.1549795
128
256
out of bounds:  Gampel  :  41.805293 , -72.2544231
384
out of bounds:  Palais-des-Congrès  :  45.5050927 , -73.5598036
out of bounds:  X-tra  :  55.8564658 , 13.0611227
512
out of bounds:  -Festplatz-am-See  :  49.52110089999999 , 6.667535999999999
out of bounds:  Rougemont  :  36.2184752 , -78.926397
out of bounds:  Wabern  :  51.1011758 , 9.350158
640
768
out of bounds:  Palace  :  36.11616850000001 , -115.174499
out of bounds:  Théâtre-La-Fourmi  :  45.823884 , 1.267253
out of bounds:  Usine  :  51.4406171 , 5.475026299999999
896
out of bounds:  Cully  :  45.565921 , -122.6066627
out of bounds:  Vernissage  :  40.9402461 , -74.1618139
1024
out of bounds:  Altstadt  :  51.2286604 , 6.772650800000001
1152
out of bounds:  Degen  :  40.70345570000001 , -73.97177959999999
out of bounds:  Paléo  :  43.2256401 , -8.3525459
out of bounds:  St.-Stephan  :  50.9272436 ,

In [11]:
ClubDataFrame.head()

,location,event,date,artists,genre,coordinates
0,Zürich,Andy-Gruber-Band,2016-04-22,['Andy Gruber Band '],traditional,"(47.3768866, 8.541694)"
1,Basel,Lilian's-White-Party,2016-04-22,['DJ Raykard '],Pop,"(47.55959860000001, 7.588576099999999)"
2,Basel,Orgelspiel-zum-Feierabend,2016-04-22,['Martin Sander '],Classic,"(47.55959860000001, 7.588576099999999)"
3,Dietikon,Feierabendmusik,2016-04-22,['Bernhard Hörler '],Classic,"(47.4053885, 8.39977)"
4,Zürich,Oomph,2016-04-22,['Oomph '],Electronic,"(47.3768866, 8.541694)"


Save the dictionary for further use

In [12]:
lu.saveDictionary(dic, "newDic")

We still have events where no coordinates could be found

In [23]:
ClubDataFrame[ClubDataFrame["coordinates"]==(None, None)].head()

,location,event,date,artists,genre,coordinates
1270,e-198060,NaN,2013-06-12,['Marco Amherd '],Classic,"(None, None)"
1308,Place-Nova-Friburgo,Estavayer-le-Lac,2013-07-31,"['Lilly Wood & The Prick ', ' Jil Is Lucky ']",NaN,"(None, None)"
3114,Schmittner-Openair,Schmitten,2013-06-22,['Animals & Friends '],Redwood,"(None, None)"
3133,Piano-di-primo-al-primo-piano,Allschwil,2013-06-22,['Gilad Atzmon & Frank Harrison '],NaN,"(None, None)"
4298,e-198144,NaN,2013-06-16,['Jimmy Cliff '],NaN,"(None, None)"


Check if (None, None) coordinates are removed

In [30]:
ClubDataFrame = ClubDataFrame[ClubDataFrame["coordinates"]!=(None, None)]
ClubDataFrame.loc[1268:1272]

,location,event,date,artists,genre,coordinates
1268,Basel,Masterrezital,2013-06-12,[' Andrea Joy Walker '],Classic,"(47.55959860000001, 7.588576099999999)"
1269,Luzern,Blue-Soul-&-Me,2013-06-12,['Mark Tenger '],NaN,"(47.05016819999999, 8.309307200000001)"
1271,Dübendorf,Christian-Muthspiel-4,2013-06-12,"[' Christian Muthspiel ', ' Matthieu Michel ',...",NaN,"(47.3971476, 8.618012799999999)"
1272,Salzhaus,Winterthur,2013-06-12,['Element Of Crime '],NaN,"(47.4975354, 8.7222841)"


Save csv with geographic coordinates

In [32]:
ClubDataFrame.to_csv("EventsChData/completeWithCoordinates.csv", index=False)

Check if write good

In [34]:
tmp = pd.read_csv("EventsChData/completeWithCoordinates.csv", index_col = False)

In [35]:
tmp.head()

,location,event,date,artists,genre,coordinates
0,Zürich,Andy-Gruber-Band,2016-04-22,['Andy Gruber Band '],traditional,"(47.3768866, 8.541694)"
1,Basel,Lilian's-White-Party,2016-04-22,['DJ Raykard '],Pop,"(47.55959860000001, 7.588576099999999)"
2,Basel,Orgelspiel-zum-Feierabend,2016-04-22,['Martin Sander '],Classic,"(47.55959860000001, 7.588576099999999)"
3,Dietikon,Feierabendmusik,2016-04-22,['Bernhard Hörler '],Classic,"(47.4053885, 8.39977)"
4,Zürich,Oomph,2016-04-22,['Oomph '],Electronic,"(47.3768866, 8.541694)"
